In [ ]:
from abc import ABC, abstractmethod
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame # apagar SparkSession em caso do uso do GlueContext

#usando glueContext para criar o spark session
from pyspark.context import SparkContext
from awsglue.context import GlueContext
import sys

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
#
spark = SparkSession.builder.appName("teste").getOrCreate() # apagar em caso do uso do GlueContext

In [ ]:
class ToolETL:
    """
    Classe responsável por processar dados da origem Old.

    Esta classe permite realizar as etapas de extração, transformação
    retornando um dataframe pyspark com os dados de origem já transformados aplicando regra de negócio.

    Attributes:
        spark (SparkSession): Sessão Spark ativa.
        file_path (str): Caminho do arquivo CSV a ser processado.
    """

    def __init__(self, spark, file_path):
        """
        Inicializa a classe com a sessão Spark e o caminho do arquivo CSV.

        Args:
            spark (SparkSession): Sessão Spark.
            file_path (str): Caminho para o arquivo CSV.
        """
        self.spark = spark
        self.file_path = file_path

    def __extract(self) -> DataFrame:
        """
        Lê dados CSV do caminho especificado.

        Returns:
            DataFrame: DataFrame contendo os dados extraídos.
        """
        return (
            self.spark.read.format("csv")
            .option("header", "true")
            .option("inferSchema", "false")
            .option("delimiter", ",")
            .load(self.file_path)
        )

    @abstractmethod
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Método abstrato para aplicar transformações no dataframe.
        """
        pass

    def run(self):
        df = self.__extract()
        df = self._transform(df)
        return df


class OrigemOLD(ToolETL):
    """
    Classe específica para tratar dados da Origem OLD.
    """

    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transformações específicas para Origem OLD.
        """
        df_transform = (
            df
            .withColumn("tipo_cliente", F.lit('J'))
            .withColumn("data_venda", F.col('data_venda').cast('date'))
            .withColumn(
                "vlr_total",
                (F.col('vlr_preco').cast('double') * F.col('qtd_vendida').cast('double')).cast('decimal(9,2)')
            )
        )
        df_transform = df_transform.filter(F.col('data_venda') <= '2025-01-25')
        return df_transform


class OrigemNEW(ToolETL):
    """
    Classe específica para tratar dados da Origem NEW.
    """

    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transformações específicas para Origem NEW.
        """
        df_transform = (
            df
            .withColumn("tipo_cliente", F.lit('J'))
            .withColumn("data_venda", F.col('data_venda').cast('date'))
            .withColumn(
                "vlr_total",
                (F.col('vlr_preco').cast('double') * F.col('qtd_vendida').cast('double')).cast('decimal(9,2)')
            )
        )
        df_transform = df_transform.filter(F.col('data_venda') >= '2025-01-25')
        return df_transform

In [175]:
origem_old = OrigemOLD(
    spark = spark, 
    file_path = 'dataset_tst.csv'
)

df_old = origem_old.run()
df_old.count()

42

In [176]:
origem_new = OrigemNEW(
    spark = spark, 
    file_path = 'dataset_tst.csv'
)

df_new = origem_new.run()
df_new.count()

12

In [179]:
df_join = df_new.join(
    df_old,
    df_new["id_pedido"] == df_old["id_pedido"],
    how="inner" # inner, right, left, full
)
df_join.toPandas().head(1000)

,id_pedido,id_produto,id_fornecedor,vlr_preco,qtd_vendida,data_venda,tipo_cliente,vlr_total,id_pedido,id_produto,id_fornecedor,vlr_preco,qtd_vendida,data_venda,tipo_cliente,vlr_total
0,1215,544,15,250.2,41,2025-01-25,J,10258.20,1215,544,15,250.2,41,2025-01-25,J,10258.20
1,1162,580,16,91.8,12,2025-01-25,J,1101.60,1162,580,16,91.8,12,2025-01-25,J,1101.60
2,1144,554,17,92.99,63,2025-01-25,J,5858.37,1144,554,17,92.99,63,2025-01-25,J,5858.37
3,1645,563,14,315.24,87,2025-01-25,J,27425.88,1645,563,14,315.24,87,2025-01-25,J,27425.88
